In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
ros= RandomOverSampler(random_state= 15)

X_ros, y_ros= ros.fit_resample(X, y)
print(f'Random Over-Sampling: {Counter(y_ros)}')

Random Over-Sampling: Counter({0: 42582, 1: 42582})


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

X_ros_train, y_ros_train= ros.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_ros= LogisticRegression(random_state=15)
lr_ros.fit(X_ros_train, y_ros_train)
y_pred_lr_ros= lr_ros.predict(X_test)
y_pred_prob_lr_ros= lr_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_ros, labels= lr_ros.classes_)
print(classification_report(y_test, y_pred_lr_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_ros))

              precision    recall  f1-score   support

           0       0.97      0.75      0.84      8517
           1       0.25      0.81      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25352597179222564
Accuracy: 0.7512991833704529
AUC: 0.8537764512255159


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_ros= DecisionTreeClassifier(random_state=15)
dt_ros.fit(X_ros_train, y_ros_train)
y_pred_dt_ros= dt_ros.predict(X_test)
y_pred_prob_dt_ros= dt_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_ros, labels= dt_ros.classes_)
print(classification_report(y_test, y_pred_dt_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_ros))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      8517
           1       0.23      0.24      0.23       912

    accuracy                           0.85      9429
   macro avg       0.57      0.58      0.57      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.22794117647058823
Accuracy: 0.8483402269593806
AUC: 0.5760387764203276


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_ros= GridSearchCV(estimator= dt_ros, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_ros.fit(X_ros_train, y_ros_train)

print("Best Parameters:", gs_dt_ros.best_params_)
print("Best Precision Score:", gs_dt_ros.best_score_)

Best Parameters: {'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Precision Score: 0.9110235517265883


In [12]:
tuned_dt_ros= gs_dt_ros.best_estimator_
y_pred_tuned_dt_ros= tuned_dt_ros.predict(X_test)
y_pred_prob_tuned_dt_ros= tuned_dt_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_ros, labels= tuned_dt_ros.classes_)
print(classification_report(y_test, y_pred_tuned_dt_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_ros))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      8517
           1       0.26      0.25      0.25       912

    accuracy                           0.86      9429
   macro avg       0.59      0.58      0.59      9429
weighted avg       0.86      0.86      0.86      9429

Precision: 0.2557077625570776
Accuracy: 0.8578852476402588
AUC: 0.5846763645052515


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_ros= KNeighborsClassifier()
knn_ros.fit(X_ros_train, y_ros_train)
y_pred_knn_ros= knn_ros.predict(X_test)
y_pred_prob_knn_ros= knn_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_ros, labels= knn_ros.classes_)
print(classification_report(y_test, y_pred_knn_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_ros))

              precision    recall  f1-score   support

           0       0.94      0.81      0.87      8517
           1       0.23      0.50      0.31       912

    accuracy                           0.78      9429
   macro avg       0.58      0.66      0.59      9429
weighted avg       0.87      0.78      0.82      9429

Precision: 0.22560078469838157
Accuracy: 0.7846006999681833
AUC: 0.7006151525638095


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_ros= GridSearchCV(estimator= knn_ros, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_ros.fit(X_ros_train, y_ros_train)
print("Best Parameters:", gs_knn_ros.best_params_)
print("Best Precision Score:", gs_knn_ros.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9179008415760157


In [15]:
tuned_knn_ros= gs_knn_ros.best_estimator_
y_pred_tuned_knn_ros= tuned_knn_ros.predict(X_test)
y_pred_prob_tuned_knn_ros= tuned_knn_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_ros, labels= tuned_knn_ros.classes_)
print(classification_report(y_test, y_pred_tuned_knn_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_ros))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      8517
           1       0.24      0.21      0.22       912

    accuracy                           0.86      9429
   macro avg       0.58      0.57      0.57      9429
weighted avg       0.85      0.86      0.86      9429

Precision: 0.24055415617128464
Accuracy: 0.8595821402057482
AUC: 0.6192331539191999


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_ros= GaussianNB()
nb_ros.fit(X_ros_train, y_ros_train)
y_pred_nb_ros= nb_ros.predict(X_test)
y_pred_prob_nb_ros= nb_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_ros, labels= nb_ros.classes_)
print(classification_report(y_test, y_pred_nb_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_ros))

              precision    recall  f1-score   support

           0       0.98      0.59      0.74      8517
           1       0.19      0.86      0.30       912

    accuracy                           0.62      9429
   macro avg       0.58      0.73      0.52      9429
weighted avg       0.90      0.62      0.70      9429

Precision: 0.18515037593984962
Accuracy: 0.6190476190476191
AUC: 0.8059134568839618


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_ros= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_ros.fit(X_ros_train, y_ros_train)
y_pred_svc_ros= svc_ros.predict(X_test)
y_pred_prob_svc_ros= svc_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_ros, labels= svc_ros.classes_)
print(classification_report(y_test, y_pred_svc_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_ros))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      8517
           1       0.16      0.03      0.05       912

    accuracy                           0.89      9429
   macro avg       0.53      0.51      0.50      9429
weighted avg       0.83      0.89      0.86      9429

Precision: 0.1566265060240964
Accuracy: 0.8911867642379891
AUC: 0.6957613410948998


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_ros= RandomForestClassifier(random_state=15)
rf_ros.fit(X_ros_train, y_ros_train)
y_pred_rf_ros= rf_ros.predict(X_test)
y_pred_prob_rf_ros= rf_ros.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_ros, labels= rf_ros.classes_)
print(classification_report(y_test, y_pred_rf_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_ros))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.37      0.16      0.23       912

    accuracy                           0.89      9429
   macro avg       0.64      0.57      0.59      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.373134328358209
Accuracy: 0.8924594336621062
AUC: 0.8207384894813057


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_ros= GridSearchCV(estimator= rf_ros, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_ros.fit(X_ros_train, y_ros_train)
print("Best Parameters:", gs_tuned_rf_ros.best_params_)
print("Best Precision Score:", gs_tuned_rf_ros.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Precision Score: 0.9586445252714368


In [21]:
tuned_rf_ros= gs_tuned_rf_ros.best_estimator_
y_pred_tuned_rf_ros= tuned_rf_ros.predict(X_test)
y_pred_prob_tuned_rf_ros= tuned_rf_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_ros, labels= tuned_rf_ros.classes_)
print(classification_report(y_test, y_pred_tuned_rf_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_ros))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.37      0.17      0.23       912

    accuracy                           0.89      9429
   macro avg       0.65      0.57      0.59      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.37438423645320196
Accuracy: 0.8924594336621062
AUC: 0.8227794926143587


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_ros= AdaBoostClassifier(random_state=15)
ada_ros.fit(X_ros_train, y_ros_train)
y_pred_ada_ros= ada_ros.predict(X_test)
y_pred_prob_ada_ros= ada_ros.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_ros, labels= ada_ros.classes_)
print(classification_report(y_test, y_pred_ada_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_ros))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.80      0.39       912

    accuracy                           0.76      9429
   macro avg       0.62      0.78      0.62      9429
weighted avg       0.90      0.76      0.80      9429

Precision: 0.25805309734513276
Accuracy: 0.7582988652030969
AUC: 0.852556818767007


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_ros= GridSearchCV(estimator= ada_ros, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_ros.fit(X_ros_train, y_ros_train)
print("Best Parameters:", gs_ada_ros.best_params_)
print("Best Precision Score:", gs_ada_ros.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 100}
Best Precision Score: 0.756103666773759


In [25]:
tuned_ada_ros= gs_ada_ros.best_estimator_
y_pred_tuned_ada_ros= tuned_ada_ros.predict(X_test)
y_pred_prob_tuned_ada_ros= tuned_ada_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_ros, labels= tuned_ada_ros.classes_)
print(classification_report(y_test, y_pred_tuned_ada_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_ros))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.80      0.39       912

    accuracy                           0.76      9429
   macro avg       0.61      0.77      0.62      9429
weighted avg       0.90      0.76      0.80      9429

Precision: 0.2565278757939308
Accuracy: 0.7569201399936366
AUC: 0.8527560462151033


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_ros= GradientBoostingClassifier(random_state=15)
grb_ros.fit(X_ros_train, y_ros_train)
y_pred_grb_ros= grb_ros.predict(X_test)
y_pred_prob_grb_ros= grb_ros.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_ros, labels= grb_ros.classes_)
print(classification_report(y_test, y_pred_grb_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_ros))

              precision    recall  f1-score   support

           0       0.97      0.74      0.84      8517
           1       0.25      0.82      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.61      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.2527137042062415
Accuracy: 0.7486477887368755
AUC: 0.8535910634870609


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_ros= GridSearchCV(estimator= grb_ros, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_ros.fit(X_ros_train, y_ros_train)

print("Best Parameters:", gs_grb_ros.best_params_)
print("Best Precision Score:", gs_grb_ros.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.7582993239785359


In [28]:
tuned_grb_ros= gs_grb_ros.best_estimator_
y_pred_tuned_grb_ros= tuned_grb_ros.predict(X_test)
y_pred_prob_tuned_grb_ros= tuned_grb_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_ros, labels= tuned_grb_ros.classes_)
print(classification_report(y_test, y_pred_tuned_grb_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_ros))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.25      0.80      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25459590704127644
Accuracy: 0.7532081875066284
AUC: 0.850115300874


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_ros= XGBClassifier(random_state=15)
xgb_ros.fit(X_ros_train, y_ros_train)
y_pred_xgb_ros= xgb_ros.predict(X_test)
y_pred_prob_xgb_ros= xgb_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_ros, labels= xgb_ros.classes_)
print(classification_report(y_test, y_pred_xgb_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_ros))

              precision    recall  f1-score   support

           0       0.95      0.81      0.88      8517
           1       0.27      0.64      0.38       912

    accuracy                           0.79      9429
   macro avg       0.61      0.73      0.63      9429
weighted avg       0.89      0.79      0.83      9429

Precision: 0.26600090785292785
Accuracy: 0.7939336090783752
AUC: 0.8118291281214661


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_ros= GridSearchCV(estimator= xgb_ros, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_ros.fit(X_ros_train, y_ros_train)

print("Best Parameters:", gs_xgb_ros.best_params_)
print("Best Precision Score:", gs_xgb_ros.best_score_)

tuned_xgb_ros= gs_xgb_ros.best_estimator_
y_pred_tuned_xgb_ros= tuned_xgb_ros.predict(X_test)
y_pred_prob_tuned_xgb_ros= tuned_xgb_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_ros, labels= tuned_xgb_ros.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_ros))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 1.0}
Best Precision Score: 0.8679043987843871
              precision    recall  f1-score   support

           0       0.93      0.87      0.90      8517
           1       0.26      0.43      0.32       912

    accuracy                           0.82      9429
   macro avg       0.59      0.65      0.61      9429
weighted avg       0.87      0.82      0.84      9429

Precision: 0.25508196721311477
Accuracy: 0.8240534521158129
AUC: 0.746054717191005


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_ros= LGBMClassifier(random_state=15)
lgm_ros.fit(X_ros_train, y_ros_train)
y_pred_lgm_ros= lgm_ros.predict(X_test)
y_pred_prob_lgm_ros= lgm_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_ros, labels= lgm_ros.classes_)
print(classification_report(y_test, y_pred_lgm_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_ros))

[LightGBM] [Info] Number of positive: 34065, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.460202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 252
[LightGBM] [Info] Number of data points in the train set: 68130, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.25      0.78      0.38       912

    accuracy                           0.75      9429
   macro avg       0.61      0.76      0.61      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25061771973173314
Accuracy: 0.7534202990773147
AUC: 0.8461210963006907


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_ros= GridSearchCV(estimator= lgm_ros, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_ros.fit(X_ros_train, y_ros_train)

print("Best Parameters:", gs_lgm_ros.best_params_)
print("Best Precision Score:", gs_lgm_ros.best_score_)

tuned_lgm_ros= gs_lgm_ros.best_estimator_
y_pred_tuned_lgm_ros= tuned_lgm_ros.predict(X_test)
y_pred_prob_tuned_lgm_ros= tuned_lgm_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_ros, labels= tuned_lgm_ros.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_ros))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 34065, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 252
[LightGBM] [Info] Number of data points in the train set: 68130, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 1.0, 'min_data_in_leaf': 50, 'num_leaves': 127, 'reg_alpha': 0.1}
Best Precision Score: 0.91356800811954
[LightGBM] [Warning] min_data_in_leaf is set=50, min

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_ros= CatBoostClassifier(random_state=15)
cat_ros.fit(X_ros_train, y_ros_train)
y_pred_cat_ros= cat_ros.predict(X_test)
y_pred_prob_cat_ros= cat_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_ros, labels= cat_ros.classes_)
print(classification_report(y_test, y_pred_cat_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_ros))

Learning rate set to 0.062484
0:	learn: 0.6725361	total: 154ms	remaining: 2m 34s
1:	learn: 0.6540983	total: 180ms	remaining: 1m 29s
2:	learn: 0.6385588	total: 211ms	remaining: 1m 10s
3:	learn: 0.6245704	total: 235ms	remaining: 58.6s
4:	learn: 0.6126651	total: 254ms	remaining: 50.5s
5:	learn: 0.6033769	total: 273ms	remaining: 45.2s
6:	learn: 0.5936972	total: 290ms	remaining: 41.2s
7:	learn: 0.5856463	total: 311ms	remaining: 38.5s
8:	learn: 0.5785540	total: 336ms	remaining: 37s
9:	learn: 0.5718428	total: 355ms	remaining: 35.2s
10:	learn: 0.5659106	total: 379ms	remaining: 34s
11:	learn: 0.5596893	total: 397ms	remaining: 32.7s
12:	learn: 0.5542418	total: 413ms	remaining: 31.4s
13:	learn: 0.5500219	total: 430ms	remaining: 30.3s
14:	learn: 0.5460643	total: 451ms	remaining: 29.6s
15:	learn: 0.5414618	total: 469ms	remaining: 28.8s
16:	learn: 0.5377960	total: 488ms	remaining: 28.2s
17:	learn: 0.5336357	total: 512ms	remaining: 27.9s
18:	learn: 0.5305846	total: 533ms	remaining: 27.5s
19:	learn: 0

164:	learn: 0.4503453	total: 3.37s	remaining: 17.1s
165:	learn: 0.4501323	total: 3.39s	remaining: 17s
166:	learn: 0.4498349	total: 3.41s	remaining: 17s
167:	learn: 0.4497099	total: 3.42s	remaining: 17s
168:	learn: 0.4493556	total: 3.44s	remaining: 16.9s
169:	learn: 0.4490248	total: 3.47s	remaining: 16.9s
170:	learn: 0.4488534	total: 3.49s	remaining: 16.9s
171:	learn: 0.4485868	total: 3.5s	remaining: 16.9s
172:	learn: 0.4484249	total: 3.52s	remaining: 16.8s
173:	learn: 0.4479830	total: 3.54s	remaining: 16.8s
174:	learn: 0.4478049	total: 3.56s	remaining: 16.8s
175:	learn: 0.4475900	total: 3.58s	remaining: 16.8s
176:	learn: 0.4471023	total: 3.6s	remaining: 16.8s
177:	learn: 0.4468640	total: 3.62s	remaining: 16.7s
178:	learn: 0.4465505	total: 3.64s	remaining: 16.7s
179:	learn: 0.4461521	total: 3.65s	remaining: 16.6s
180:	learn: 0.4459464	total: 3.67s	remaining: 16.6s
181:	learn: 0.4457073	total: 3.69s	remaining: 16.6s
182:	learn: 0.4454575	total: 3.71s	remaining: 16.5s
183:	learn: 0.445098

328:	learn: 0.4107819	total: 6.35s	remaining: 13s
329:	learn: 0.4106007	total: 6.37s	remaining: 12.9s
330:	learn: 0.4105049	total: 6.38s	remaining: 12.9s
331:	learn: 0.4104148	total: 6.4s	remaining: 12.9s
332:	learn: 0.4101172	total: 6.42s	remaining: 12.9s
333:	learn: 0.4099780	total: 6.44s	remaining: 12.8s
334:	learn: 0.4097831	total: 6.46s	remaining: 12.8s
335:	learn: 0.4095441	total: 6.47s	remaining: 12.8s
336:	learn: 0.4093359	total: 6.49s	remaining: 12.8s
337:	learn: 0.4091120	total: 6.51s	remaining: 12.7s
338:	learn: 0.4089362	total: 6.52s	remaining: 12.7s
339:	learn: 0.4087741	total: 6.54s	remaining: 12.7s
340:	learn: 0.4086298	total: 6.57s	remaining: 12.7s
341:	learn: 0.4084001	total: 6.58s	remaining: 12.7s
342:	learn: 0.4082037	total: 6.6s	remaining: 12.6s
343:	learn: 0.4080924	total: 6.62s	remaining: 12.6s
344:	learn: 0.4079779	total: 6.63s	remaining: 12.6s
345:	learn: 0.4076794	total: 6.65s	remaining: 12.6s
346:	learn: 0.4074622	total: 6.67s	remaining: 12.6s
347:	learn: 0.40

491:	learn: 0.3802463	total: 9.33s	remaining: 9.63s
492:	learn: 0.3800299	total: 9.35s	remaining: 9.61s
493:	learn: 0.3798229	total: 9.36s	remaining: 9.59s
494:	learn: 0.3796427	total: 9.38s	remaining: 9.57s
495:	learn: 0.3794538	total: 9.4s	remaining: 9.55s
496:	learn: 0.3793382	total: 9.41s	remaining: 9.53s
497:	learn: 0.3791920	total: 9.43s	remaining: 9.51s
498:	learn: 0.3790102	total: 9.45s	remaining: 9.48s
499:	learn: 0.3788187	total: 9.46s	remaining: 9.46s
500:	learn: 0.3785921	total: 9.48s	remaining: 9.44s
501:	learn: 0.3783937	total: 9.5s	remaining: 9.42s
502:	learn: 0.3782262	total: 9.52s	remaining: 9.4s
503:	learn: 0.3780351	total: 9.54s	remaining: 9.39s
504:	learn: 0.3779109	total: 9.56s	remaining: 9.37s
505:	learn: 0.3777917	total: 9.57s	remaining: 9.35s
506:	learn: 0.3775819	total: 9.59s	remaining: 9.33s
507:	learn: 0.3774180	total: 9.61s	remaining: 9.31s
508:	learn: 0.3772198	total: 9.63s	remaining: 9.29s
509:	learn: 0.3771132	total: 9.64s	remaining: 9.26s
510:	learn: 0.3

653:	learn: 0.3548168	total: 13s	remaining: 6.86s
654:	learn: 0.3545415	total: 13s	remaining: 6.85s
655:	learn: 0.3543564	total: 13s	remaining: 6.83s
656:	learn: 0.3542205	total: 13.1s	remaining: 6.82s
657:	learn: 0.3540723	total: 13.1s	remaining: 6.8s
658:	learn: 0.3539654	total: 13.1s	remaining: 6.78s
659:	learn: 0.3538094	total: 13.1s	remaining: 6.77s
660:	learn: 0.3536608	total: 13.2s	remaining: 6.75s
661:	learn: 0.3535525	total: 13.2s	remaining: 6.73s
662:	learn: 0.3533876	total: 13.2s	remaining: 6.72s
663:	learn: 0.3532916	total: 13.2s	remaining: 6.7s
664:	learn: 0.3531655	total: 13.3s	remaining: 6.68s
665:	learn: 0.3530602	total: 13.3s	remaining: 6.67s
666:	learn: 0.3529050	total: 13.3s	remaining: 6.65s
667:	learn: 0.3528108	total: 13.3s	remaining: 6.63s
668:	learn: 0.3526848	total: 13.4s	remaining: 6.61s
669:	learn: 0.3525586	total: 13.4s	remaining: 6.6s
670:	learn: 0.3524512	total: 13.4s	remaining: 6.58s
671:	learn: 0.3522834	total: 13.4s	remaining: 6.57s
672:	learn: 0.3521044

812:	learn: 0.3349505	total: 16.4s	remaining: 3.77s
813:	learn: 0.3348486	total: 16.4s	remaining: 3.75s
814:	learn: 0.3347620	total: 16.4s	remaining: 3.73s
815:	learn: 0.3346237	total: 16.5s	remaining: 3.71s
816:	learn: 0.3345709	total: 16.5s	remaining: 3.69s
817:	learn: 0.3344581	total: 16.5s	remaining: 3.67s
818:	learn: 0.3343605	total: 16.5s	remaining: 3.65s
819:	learn: 0.3342530	total: 16.5s	remaining: 3.63s
820:	learn: 0.3341440	total: 16.5s	remaining: 3.61s
821:	learn: 0.3340835	total: 16.6s	remaining: 3.58s
822:	learn: 0.3339636	total: 16.6s	remaining: 3.56s
823:	learn: 0.3338454	total: 16.6s	remaining: 3.54s
824:	learn: 0.3337325	total: 16.6s	remaining: 3.52s
825:	learn: 0.3335990	total: 16.6s	remaining: 3.5s
826:	learn: 0.3334730	total: 16.6s	remaining: 3.48s
827:	learn: 0.3333323	total: 16.7s	remaining: 3.46s
828:	learn: 0.3332357	total: 16.7s	remaining: 3.44s
829:	learn: 0.3331383	total: 16.7s	remaining: 3.42s
830:	learn: 0.3330536	total: 16.7s	remaining: 3.4s
831:	learn: 0.

978:	learn: 0.3153373	total: 19.4s	remaining: 415ms
979:	learn: 0.3152013	total: 19.4s	remaining: 395ms
980:	learn: 0.3149698	total: 19.4s	remaining: 376ms
981:	learn: 0.3148518	total: 19.4s	remaining: 356ms
982:	learn: 0.3147367	total: 19.4s	remaining: 336ms
983:	learn: 0.3146385	total: 19.4s	remaining: 316ms
984:	learn: 0.3145356	total: 19.5s	remaining: 296ms
985:	learn: 0.3143804	total: 19.5s	remaining: 277ms
986:	learn: 0.3142500	total: 19.5s	remaining: 257ms
987:	learn: 0.3141337	total: 19.5s	remaining: 237ms
988:	learn: 0.3140457	total: 19.5s	remaining: 217ms
989:	learn: 0.3139653	total: 19.5s	remaining: 197ms
990:	learn: 0.3137893	total: 19.6s	remaining: 178ms
991:	learn: 0.3137125	total: 19.6s	remaining: 158ms
992:	learn: 0.3136383	total: 19.6s	remaining: 138ms
993:	learn: 0.3135423	total: 19.6s	remaining: 118ms
994:	learn: 0.3134538	total: 19.6s	remaining: 98.7ms
995:	learn: 0.3133806	total: 19.6s	remaining: 78.9ms
996:	learn: 0.3132492	total: 19.7s	remaining: 59.2ms
997:	lear

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_ros= GridSearchCV(estimator= cat_ros, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_ros.fit(X_ros_train, y_ros_train)

print("Best Parameters:", gs_cat_ros.best_params_)
print("Best Precision Score:", gs_cat_ros.best_score_)

tuned_cat_ros= gs_cat_ros.best_estimator_
y_pred_tuned_cat_ros= tuned_cat_ros.predict(X_test)
y_pred_prob_tuned_cat_ros= tuned_cat_ros.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_ros, labels= tuned_cat_ros.classes_)
print(classification_report(y_test, y_pred_tuned_cat_ros, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_ros, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_ros))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_ros))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.5019810	total: 54.1ms	remaining: 5.36s
1:	learn: 0.4650811	total: 112ms	remaining: 5.48s
2:	learn: 0.4472011	total: 163ms	remaining: 5.27s
3:	learn: 0.4335853	total: 210ms	remaining: 5.04s
4:	learn: 0.4121218	total: 272ms	remaining: 5.16s
5:	learn: 0.3935136	total: 337ms	remaining: 5.27s
6:	learn: 0.3798147	total: 402ms	remaining: 5.35s
7:	learn: 0.3633401	total: 458ms	remaining: 5.27s
8:	learn: 0.3497982	total: 525ms	remaining: 5.3s
9:	learn: 0.3372596	total: 583ms	remaining: 5.24s
10:	learn: 0.3289305	total: 640ms	remaining: 5.17s
11:	learn: 0.3111677	total: 706ms	remaining: 5.18s
12:	learn: 0.2920249	total: 776ms	remaining: 5.19s
13:	learn: 0.2806651	total: 836ms	remaining: 5.13s
14:	learn: 0.2664728	total: 899ms	remaining: 5.09s
15:	learn: 0.2528652	total: 952ms	remaining: 5s
16:	learn: 0.2446943	total: 1.01s	remaining: 4.95s
17:	learn: 0.2293079	total: 1.07s	remaining: 4.9s
18:	learn: 0.2168316	total: 1.13s	

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_ros, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_ros, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_ros, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_ros, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_ros, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_ros, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_ros, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_ros, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_ros, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_ros, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_ros, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_ros, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_ros, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_ros, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_ros, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_ros, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_ros, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_ros, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_ros, zero_division= 0))

Logistic Regression Precision: 0.25352597179222564
Decision Tree Precision: 0.22794117647058823
Tuned Decision Tree Precision: 0.2557077625570776
KNeighborsClassifier Precision: 0.22560078469838157
Tuned KNeighborsClassifier Precision: 0.24055415617128464
GaussianNB Precision: 0.18515037593984962
SVM Precision: 0.1566265060240964
Random Forest Precision: 0.373134328358209
Tuned Random Forest Precision: 0.37438423645320196
AdaBoost Precision: 0.25805309734513276
Tuned AdaBoost Precision: 0.2565278757939308
GradientBoosting Precision: 0.2527137042062415
Tuned GradientBoosting Precision: 0.25459590704127644
XGB Precision: 0.26600090785292785
Tuned XGB Precision: 0.25508196721311477
LGBM Precision: 0.25061771973173314
Tuned LGBM Precision: 0.27528675703858185
CatBoost Precision: 0.27438468550592526
Tuned CatBoost Precision: 0.2611607142857143


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_ros, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_ros, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_ros, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_ros, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_ros, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_ros, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_ros, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_ros, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_ros, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_ros, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_ros, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_ros, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_ros, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_ros, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_ros, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_ros, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_ros, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_ros, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_ros, zero_division= 0)
                  }

ros_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
ros_precision= ros_precision.sort_values(by= 'Precision Score', ascending=False)
print(ros_precision)

                                    Model  Precision Score
8          Tuned Random Forest Precision:         0.374384
7                Random Forest Precision:         0.373134
16                  Tuned LGBM Precision:         0.275287
17                    CatBoost Precision:         0.274385
13                         XGB Precision:         0.266001
18              Tuned CatBoost Precision:         0.261161
9                     AdaBoost Precision:         0.258053
10              Tuned AdaBoost Precision:         0.256528
2          Tuned Decision Tree Precision:         0.255708
14                   Tuned XGB Precision:         0.255082
12      Tuned GradientBoosting Precision:         0.254596
0          Logistic Regression Precision:         0.253526
11            GradientBoosting Precision:         0.252714
15                        LGBM Precision:         0.250618
4   Tuned KNeighborsClassifier Precision:         0.240554
1                Decision Tree Precision:         0.2279

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_ros),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_ros),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_ros),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_ros),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_ros),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_ros),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_ros),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_ros),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_ros),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_ros),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_ros),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_ros),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_ros),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_ros),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_ros),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_ros),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_ros),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_ros),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_ros)
                  }

ros_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
ros_accuracy= ros_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(ros_accuracy)

                                   Model  Accuracy Score
7                Random Forest Accuracy:        0.892459
8          Tuned Random Forest Accuracy:        0.892459
6                          SVM Accuracy:        0.891187
4   Tuned KNeighborsClassifier Accuracy:        0.859582
18              Tuned CatBoost Accuracy:        0.857885
2          Tuned Decision Tree Accuracy:        0.857885
16                  Tuned LGBM Accuracy:        0.857567
1                Decision Tree Accuracy:        0.848340
14                   Tuned XGB Accuracy:        0.824053
17                    CatBoost Accuracy:        0.798282
13                         XGB Accuracy:        0.793934
3         KNeighborsClassifier Accuracy:        0.784601
9                     AdaBoost Accuracy:        0.758299
10              Tuned AdaBoost Accuracy:        0.756920
15                        LGBM Accuracy:        0.753420
12      Tuned GradientBoosting Accuracy:        0.753208
0          Logistic Regression 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_ros),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_ros),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_ros),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_ros),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_ros),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_ros),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_ros),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_ros),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_ros),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_ros),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_ros),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_ros),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_ros),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_ros),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_ros),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_ros),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_ros),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_ros),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_ros)
                  }

ros_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
ros_auc= ros_auc.sort_values(by= 'AUC Score', ascending=False)
print(ros_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.853776
11            GradientBoosting AUC:   0.853591
10              Tuned AdaBoost AUC:   0.852756
9                     AdaBoost AUC:   0.852557
12      Tuned GradientBoosting AUC:   0.850115
15                        LGBM AUC:   0.846121
17                    CatBoost AUC:   0.826335
8          Tuned Random Forest AUC:   0.822779
7                Random Forest AUC:   0.820738
13                         XGB AUC:   0.811829
5                   GaussianNB AUC:   0.805913
16                  Tuned LGBM AUC:   0.751320
14                   Tuned XGB AUC:   0.746055
18              Tuned CatBoost AUC:   0.743450
3         KNeighborsClassifier AUC:   0.700615
6                          SVM AUC:   0.695761
4   Tuned KNeighborsClassifier AUC:   0.619233
2          Tuned Decision Tree AUC:   0.584676
1                Decision Tree AUC:   0.576039
